function

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///./database/eleclog_copy.db

'Connected: @./database/eleclog_copy.db'

In [3]:
readings_orig = %sql select * from readings order by timestamp;

 * sqlite:///./database/eleclog_copy.db
Done.


In [4]:
%sql SELECT sql FROM sqlite_master WHERE name = 'readings';

 * sqlite:///./database/eleclog_copy.db
Done.


sql
"CREATE TABLE readings (timestamp DATETIME, demand NUMERIC, demand_max NUMERIC, anm_generated NUMERIC, non_anm_generated NUMERIC, total_generation_capacity NUMERIC)"


In [5]:
%sql sqlite:///./database/ANMlog_copy.db

'Connected: @./database/ANMlog_copy.db'

In [6]:
%sql SELECT COUNT(*) FROM records;

 * sqlite:///./database/ANMlog_copy.db
   sqlite:///./database/eleclog_copy.db
Done.


COUNT(*)
15711


In [7]:
records = %sql select * from records order by timestamp;

 * sqlite:///./database/ANMlog_copy.db
   sqlite:///./database/eleclog_copy.db
Done.


In [8]:
print(readings_orig[0], records[0])

('2019-01-16 14:36:03', 23.22, 35.7, 12.478, 17.782, 57.099999999999994) ('2019-01-16 22:42:33', '{"2": ["warning", "ok", "ok"], "2B": ["warning", "ok", "ok"], "2A": ["remove", "ok", "ok"], "4A": ["ok", "ok", "ok"], "3": ["ok", "ok", "ok"], "Core": ["ok", "ok", "ok"], "1": ["ok", "ok", "ok"], "4": ["ok", "ok", "ok"], "1A": ["ok", "ok", "ok"]}')


In [9]:
import json

In [10]:
record = json.loads(records[0]['log'])
record

{'2': ['warning', 'ok', 'ok'],
 '2B': ['warning', 'ok', 'ok'],
 '2A': ['remove', 'ok', 'ok'],
 '4A': ['ok', 'ok', 'ok'],
 '3': ['ok', 'ok', 'ok'],
 'Core': ['ok', 'ok', 'ok'],
 '1': ['ok', 'ok', 'ok'],
 '4': ['ok', 'ok', 'ok'],
 '1A': ['ok', 'ok', 'ok']}

In [11]:
%sql SELECT sql FROM sqlite_master WHERE name = 'records';

 * sqlite:///./database/ANMlog_copy.db
   sqlite:///./database/eleclog_copy.db
Done.


sql
"CREATE TABLE records(timestamp DATETIME, log TEXT)"


In [12]:
%%sql
select 
    timestamp as start,
    lead ( timestamp, 1, 0 ) over timestamp as end,
    log as data
from records limit 10 ;

 * sqlite:///./database/ANMlog_copy.db
   sqlite:///./database/eleclog_copy.db
(sqlite3.OperationalError) near "timestamp": syntax error
[SQL: select 
    timestamp as start,
    lead ( timestamp, 1, 0 ) over timestamp as end,
    log as data
from records limit 10 ;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [13]:
%%sql
select * from records where timestamp < '2019-01-17 09:35:11' order by timestamp desc limit 2;


 * sqlite:///./database/ANMlog_copy.db
   sqlite:///./database/eleclog_copy.db
Done.


timestamp,log
2019-01-17 09:34:10,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
2019-01-17 09:31:11,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""remove"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"


In [14]:
%sql select count(*) from readings;

 * sqlite:///./database/ANMlog_copy.db
   sqlite:///./database/eleclog_copy.db
(sqlite3.OperationalError) no such table: readings
[SQL: select count(*) from readings;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [15]:
%sql sqlite:///./database/combined.db

'Connected: @./database/combined.db'

%sql CREATE TABLE readings (timestamp DATETIME, demand NUMERIC, demand_max NUMERIC, anm_generated NUMERIC, non_anm_generated NUMERIC, total_generation_capacity NUMERIC);

%sql CREATE TABLE records(timestamp DATETIME, log TEXT);

In [16]:
records.DataFrame().head()


,timestamp,log
0,2019-01-16 22:42:33,"{""2"": [""warning"", ""ok"", ""ok""], ""2B"": [""warning..."
1,2019-01-16 22:46:25,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."
2,2019-01-16 22:59:22,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."
3,2019-01-16 23:03:05,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."
4,2019-01-16 23:06:26,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."


In [17]:
records = records.DataFrame()
#%sql persist records

In [18]:
readings = readings_orig.DataFrame()

In [22]:
# %sql persist readings

   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db


'Persisted readings'

In [23]:
%%sql
select
    *
from
    readings
left join
    records
where
    substr(readings.timestamp, 0, 17) == substr(records.timestamp, 0, 17)
limit
    10
    ;

   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.


index,timestamp,demand,demand_max,anm_generated,non_anm_generated,total_generation_capacity,index_1,timestamp_1,log
470,2019-01-16 22:42:03,19.66,35.7,17.145,17.075,57.099999999999994,0,2019-01-16 22:42:33,"{""2"": [""warning"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""], ""1"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""]}"
474,2019-01-16 22:46:03,18.7,35.7,14.572,16.457,57.1,1,2019-01-16 22:46:25,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
487,2019-01-16 22:59:03,18.28,35.7,15.618,16.093,57.099999999999994,2,2019-01-16 22:59:22,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
491,2019-01-16 23:03:05,18.56,35.7,13.922,18.143,57.1,3,2019-01-16 23:03:05,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
494,2019-01-16 23:06:03,18.2,35.7,16.457,17.996,57.099999999999994,4,2019-01-16 23:06:26,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
1106,2019-01-17 09:18:05,23.16,35.7,15.016,17.61,57.1,5,2019-01-17 09:18:10,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""warning"", ""ok"", ""ok""], ""2B"": [""ok"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
1107,2019-01-17 09:19:05,23.62,35.7,16.485,17.773,57.099999999999994,6,2019-01-17 09:19:10,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""ok"", ""ok"", ""ok""], ""2B"": [""ok"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
1109,2019-01-17 09:21:05,23.43,35.7,15.955,18.045,57.1,7,2019-01-17 09:21:11,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""ok"", ""ok"", ""ok""], ""2B"": [""ok"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
1115,2019-01-17 09:27:05,24.03,35.7,15.882,17.89,57.1,8,2019-01-17 09:27:10,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""remove"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
1117,2019-01-17 09:29:05,23.45,35.7,15.7,17.904,57.099999999999994,9,2019-01-17 09:29:11,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", "

In [20]:
for reading in readings_orig[30000:30010]:
#reading = readings[1005]
    readingdate= "'"+reading['timestamp']+"'"
    anmstatus = %sql select * from records where timestamp < $readingdate order by timestamp desc limit 1;
    print(reading['timestamp'], anmstatus[0]['timestamp'])

   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:14:09 2019-02-06 18:13:09
   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:15:05 2019-02-06 18:14:13
   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:16:05 2019-02-06 18:14:13
   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:17:05 2019-02-06 18:14:13
   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:18:05 2019-02-06 18:14:13
   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:19:05 2019-02-06 18:18:10
   sqlite:///./database/ANMlog_cop

In [21]:
for reading in readings_orig[30000:30010]:
#reading = readings[1005]
    readingdate= "'"+reading['timestamp']+"'"
    anmrecord = %sql select * from records where substr(timestamp, 0, 17) <= substr($readingdate, 0, 17) order by timestamp desc limit 1;
    anmstatus = json.loads(anmrecord[0]['log'])
    print(reading['timestamp'],
          reading['demand'],
          reading['demand_max'],
          reading['anm_'],
          reading[''],
          reading[''],
          reading[''],
          reading[''],
          reading[''],
          anmrecord[0]['timestamp'],
          anmstatus['Core'][0],
          anmstatus['Core'][1],
          anmstatus['Core'][2],
          anmstatus['1'][0],
          anmstatus['1'][1],
          anmstatus['1'][2],
          anmstatus['2'][0],
          anmstatus['2'][1],
          anmstatus['2'][2],
          anmstatus['2A'][0],
          anmstatus['2A'][1],
          anmstatus['2A'][2],
          anmstatus['2B'][0],
          anmstatus['2B'][1],
          anmstatus['2B'][2],
          anmstatus['3'][0],
          anmstatus['3'][1],
          anmstatus['3'][2],
          anmstatus['4'][0],
          anmstatus['4'][1],
          anmstatus['4'][2],
          anmstatus['4A'][0],
          anmstatus['4A'][1],
          anmstatus['4A'][2])
    
    

   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:14:09 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:15:05 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:16:05 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:17:05 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
   sqlite:///./database/ANMl